In [27]:
from sdv.demo import load_timeseries_demo

ModuleNotFoundError: No module named 'copulas'

In [23]:
def trends_only(source_df: pd.DataFrame, trend_col: str) -> (float, pd.DataFrame):
    """ Extract trends as training features vs total volume """
    df = source_df.copy()
    start_val = df.at[0, trend_col]
    df.sin = df[[trend_col]].diff()
    df.at[0, trend_col] = 0.00
    return start_val, df


def restore_daily(source_df: pd.DataFrame, start_val: float, trend_col: str):
    """ Restore daily cumulative values from trend data """
    df = source_df.copy()
    df.at[0, trend_col] = start_val
    df[trend_col] = df[trend_col].cumsum()
    df[trend_col] = df[trend_col].apply(pd.to_numeric, downcast='float', errors='coerce').round(2)
    df.dropna(inplace=True)
    return df

# Extract trends from timeseries column to create training set
start_val, trends_df = trends_only(train_df, trend_col)
trends_df

NameError: name 'trend_col' is not defined

In [24]:
# Create the Gretel Synthtetics Training / Model Configuration

from pathlib import Path
from gretel_helpers.synthetics import create_df, SyntheticDataBundle

config_template = {
    "epochs": 50,
    "early_stopping": False,
    "vocab_size": 20,
    "reset_states": False, 
    "checkpoint_dir": str(Path.cwd() / "checkpoints-sin"),
    "overwrite": True,
}

model = SyntheticDataBundle(
    training_df=trends_df,
    delimiter=",",
    auto_validate=True, 
    synthetic_config=config_template,
    header_prefix=['date'] # Seed the model with the date field
    )

model.build()
model.train()

ModuleNotFoundError: No module named 'gretel_helpers'

In [19]:
#Set up the seed fields
seed_fields = ["date", "district_id"]

task = {
    'type': 'seed',
    'attrs': {
        'fields': seed_fields,
    }
}

# Fine tune model parameters. These are the parameters we found to work best.  This is "Run 20" in the document
config['models'][0]['synthetics']['task'] = task

config['models'][0]['synthetics']['params']['vocab_size'] = 19
config['models'][0]['synthetics']['params']['learning_rate'] = 0.001
config['models'][0]['synthetics']['params']['epochs'] = 500
config['models'][0]['synthetics']['params']['dropout_rate'] = .5
config['models'][0]['synthetics']['params']['gen_temp'] = .8
config['models'][0]['synthetics']['generate']['num_records'] = train_df.shape[0]

TypeError: string indices must be integers